In [25]:
import itertools
from collections import deque

import numpy as np

In [26]:
def parse_input(path):
    with open(path) as file_in:
        data = file_in.read().splitlines()

    data = [tuple(map(int, row.split(","))) for row in data]
    data = [(y, x) for x, y in data]

    return data

In [27]:
def compute_area(t1, t2):
    length_vertical = abs(t2[0] - t1[0]) + 1
    length_horizontal = abs(t2[1] - t1[1]) + 1
    return length_vertical*length_horizontal

In [28]:
def main1(input_file):
    red_tiles = parse_input(input_file)

    max_area = 0
    for t1, t2 in itertools.combinations(red_tiles, 2):
        max_area = max(max_area, compute_area(t1, t2))

    return max_area

In [29]:
def build_grid_compressed(red_tiles_compressed):

    # Add red tiles
    coords = red_tiles_compressed
    red_or_green_tiles = set(coords)

    # Add green tiles between red tiles
    for tile in list(range(0, len(coords)-1, 2)) + list(range(1, len(coords)-1, 2)):
        if coords[tile][0] == coords[tile+1][0]:
            # Horizontal fill
            for col in range(min(coords[tile][1], coords[tile+1][1])+1, max(coords[tile][1], coords[tile+1][1])):
                red_or_green_tiles.add((coords[tile][0], col))
        else:
            # Vertical fill
            for row in range(min(coords[tile][0], coords[tile+1][0])+1, max(coords[tile][0], coords[tile+1][0])):
                red_or_green_tiles.add((row, coords[tile][1]))

    # Fill the interior with green tiles
    start = (124, 1)
    queue = deque([start])
    visited = set()
    interior = set()
    interior.add(start)

    max_row = max(coords, key=lambda t: t[0])[0]
    max_col = max(coords, key=lambda t: t[1])[1]

    while queue:
        pos = queue.popleft()
        if pos not in red_or_green_tiles and pos not in interior:
            interior.add(pos)
        for neigh in [(pos[0], pos[1]-1), (pos[0], pos[1]+1), (pos[0]-1, pos[1]), (pos[0]+1, pos[1])]:
            if 0 <= neigh[0] < max_row and 0 <= neigh[1] < max_col:
                if neigh not in red_or_green_tiles and neigh not in visited:
                    queue.append(neigh)
                    visited.add(neigh)

    red_or_green_tiles = red_or_green_tiles | interior

    grid_compressed = np.zeros((max_row+1, max_col+1))
    for row, col in red_or_green_tiles:
        grid_compressed[row, col] = 1

    return grid_compressed

In [30]:
def compute_area_if_valid(t1, t2, grid):
    area = compute_area(t1, t2)

    min_row, max_row = min(t1[0], t2[0]), max(t1[0], t2[0])
    min_col, max_col = min(t1[1], t2[1]), max(t1[1], t2[1])
    grid_rectangle = grid[min_row:max_row+1, min_col:max_col+1]

    if grid_rectangle.sum() == area:
        return area
    else:
        return 0

In [31]:
def main2(input_file):
    red_tiles = parse_input(input_file)

    # Coordinates compression
    row_coords = sorted(list(set((t[0] for t in red_tiles))))
    row_coords_mapping = {coord_init: row_coords.index(coord_init) for coord_init in row_coords}
    col_coords = sorted(list(set((t[1] for t in red_tiles))))
    col_coords_mapping = {coord_init: col_coords.index(coord_init) for coord_init in col_coords}
    red_tiles_compressed = [(row_coords_mapping[row], col_coords_mapping[col]) for row, col in red_tiles]

    # Build grid
    grid = build_grid_compressed(red_tiles_compressed)

    # Compute max area of valid rectangles
    max_area = 0
    rectangle_with_max_area_compressed = None
    for t1, t2 in itertools.combinations(red_tiles_compressed, 2):
        area = compute_area_if_valid(t1, t2, grid)
        if area > max_area:
            max_area = max(max_area, area)
            rectangle_with_max_area_compressed = (t1, t2)

    t1_comp, t2_comp = rectangle_with_max_area_compressed
    t1_uncomp = red_tiles[red_tiles_compressed.index(t1_comp)]
    t2_uncomp = red_tiles[red_tiles_compressed.index(t2_comp)]
    area_uncomp = compute_area(t1_uncomp, t2_uncomp)

    return area_uncomp

In [32]:
assert main1("example.txt") == 50
main1("input.txt")

4741848414

In [33]:
main2("input.txt")

1508918480